# Problem statement

## The experiment focuses on testing the Dictionary Learning method on a single image to do a placeholder test and use it later in pipeline

## Imports

In [ ]:
import sys
sys.path.append('.')
sys.path.append('..')
from utils.noise_generation import add_noise_and_save
from utils.denoising_algorithms import denoise_image_with_dictionary_learning
from utils.metrics import calculate_metrics
import os
from skimage import io

## Constants and handling

In [ ]:
# Change the current working directory to one level up from the current directory.
# This is often used in notebook environments like Jupyter to navigate the file system.
%cd ../

# Retrieve the current working directory, which is now set to the parent directory of the previous one.
# The HOME variable is used as a reference point in the file system to build paths relative to the project's root.
HOME = os.getcwd()

# Change the directory to the 'experiments' subdirectory from the current location.
# This is typically done to ensure that the execution context is correctly set for subsequent operations,
# especially when the code is dependent on the working directory.
%cd experiments

# Configure the path settings for the image processing task:

# image_directory: Specifies the path to the directory containing the original images to be processed.
# It is constructed dynamically using the HOME variable to ensure the path is adaptable to different environments.
image_directory = os.path.join(HOME, 'data/original_photos')

# main_folder: Defines the main directory where processed images (noisy images in this case) will be stored.
# Like image_directory, it is built using the HOME variable, allowing flexibility in the file structure.
main_folder = os.path.join(HOME, 'data')

# noise_type: A string that specifies the type of noise to be applied to the images. This is used to
# differentiate between various noise application methods and to store the resulting images accordingly.
noise_type = 'gaussian'

# noise_image_directory: The directory where the noisy images will be saved. It is structured to organize
# images by the type of noise applied, aiding in the systematic storage and retrieval of processed images.
noise_image_directory = os.path.join(HOME, f'data/noise_photos/{noise_type}')

# variance: A floating-point value that specifies the intensity of the noise to be added to the images.
# This parameter can directly affect the visual characteristics of the resultant noisy images.
variance = 0.1

# n_components: Specifies the number of principal components to be retained in the PCA process when
# denoising the images. It plays a critical role in determining the balance between noise removal and
# detail preservation in the denoised images.
n_components = 500


## Add to images noise and save them

In [ ]:
# Loop through all files in the specified directory
for filename in os.listdir(image_directory):
    # Construct the full file path
    file_path = os.path.join(image_directory, filename)
    # Check if the file is an image
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Apply the noise and save the image
        add_noise_and_save(file_path, main_folder, noise_type, variance)
        print(f"Processed {filename}")

## Denoise previously noised images using Dictionary Learning

In [ ]:
# Iterate through each file in the specified directory.
# This loop is designed to process each file found in the directory where the noisy images are stored.
for filename in os.listdir(noise_image_directory):
    # Construct the full path to the file by combining the directory path with the filename.
    # This provides a complete path to access the file for processing.
    file_path = os.path.join(noise_image_directory, filename)
    # Check if the current file is an image, specifically looking for common image file extensions.
    # This ensures that the script only attempts to process image files, ignoring others.
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load the image in grayscale, which simplifies the PCA process by reducing the data complexity.
        # This is a common approach when applying PCA for denoising, as it works on a single channel.
        image = io.imread(file_path, as_gray=True)
        print(f"Image shape: {image.shape}")
        # Attempt to apply PCA denoising to the image, encapsulated in a try-except block to handle potential errors.
        try:
            denoise_image_with_dictionary_learning(file_path, main_folder, noise_type, variance, n_components)
            print(f"Denoising completed for {filename}")  # Indicate successful denoising of the image.
        except ValueError as e:
            # Catch and report any errors encountered during the denoising process, typically related to PCA parameters.
            print(f"Error processing {filename}: {e}")


## Metrics calculation and algorithm evaluation

In [ ]:
ssim, psnr = calculate_metrics('', '')